粗糙集是波兰理工大学Z.pawlak教授提出用来研究不完整数据，不精确知识的表达、学习，归纳等的一套理论

它是一种新的处理模糊和不确定性问题的数学工具，已被广泛应用于知识发现、机器学习、决策支持、模式识别、专家系统及归纳推理等领域。

粗糙集理论的特点是能够分析隐藏在数据中的事实，又不需要关于数据附加信息。

其主要思想是在保持分类能力不变的前提下，通过知识约简，导出问题的决策或分类规则。

从数学的角度看，粗糙集是研究集合的；从编程的角度看，粗糙集的研究对象是矩阵,只不过是一些特殊的矩阵；从人工智能的角度来看，粗糙集研究的是决策表。
##### 视频讲解粗糙集和属性约简

* [视频讲解粗糙集1](https://www.bilibili.com/video/BV1xu41157ss/?spm_id_from=333.999.0.0&vd_source=ce2c265fe654b727a504cf64875b6105)

* [视频讲解粗糙集2](https://www.bilibili.com/video/BV1D14y1R7qB/?spm_id_from=333.788.recommend_more_video.-1&vd_source=ce2c265fe654b727a504cf64875b6105)

* [视频讲解粗糙集3](https://www.bilibili.com/video/BV18z4y1J7GF/?spm_id_from=333.788.recommend_more_video.0&vd_source=ce2c265fe654b727a504cf64875b6105)
##### 文字讲解什么是粗糙集
* [什么是粗糙集1](https://www.jianshu.com/p/a129b7a6be9e)

* [什么是粗糙集2](https://www.jianshu.com/p/bb61d6bc81e1)

* [什么是粗糙集3](https://www.jianshu.com/p/ab3135fd5d40)

* [什么是粗糙集4](https://www.jianshu.com/p/b48d46688078)

* [什么是粗糙集5](https://www.jianshu.com/p/768166d73aa1)

* [什么是粗糙集6](https://www.jianshu.com/p/b303f95313cf)

In [1]:
import numpy as np
import pandas as pd


In [2]:
table = pd.DataFrame(
    data=np.matrix([["e1",'是', '是', '正常', '否']
    ,["e2",'是', '是', '高', '是']
    ,["e3",'是', '是', '很高', '是']
    ,["e4",'否', '是', '正常', '否']
    ,["e5",'否', '否', '高', '否']
    ,["e6",'否', '是', '很高', '是']])
    ,columns= ["病人","头疼","肌肉疼","体温","流感"]
    # ,index = ['e1','e2','e3','e4','e5','e6']
        )
table

,病人,头疼,肌肉疼,体温,流感
0,e1,是,是,正常,否
1,e2,是,是,高,是
2,e3,是,是,很高,是
3,e4,否,是,正常,否
4,e5,否,否,高,否
5,e6,否,是,很高,是


#### 决策属性 $ D $
* 决策属性其实就相当于平时分类建模时候训练数据的标签
* 在这里决策属性其实就是 “流感” 这一列

In [3]:
D = np.ravel(table.iloc[:,-1])
D = np.array(D,dtype=np.str_)

#### 条件属性 $ C $
* 条件属性就是去掉索引和标签的部分

In [4]:
C = np.matrix(table.iloc[:,1:-1],dtype=np.str_)
C 

matrix([['是', '是', '正常'],
        ['是', '是', '高'],
        ['是', '是', '很高'],
        ['否', '是', '正常'],
        ['否', '否', '高'],
        ['否', '是', '很高']], dtype='<U2')

#### $ R $ 其实就是个属性值的列名

In [5]:
R = np.array(table.columns,dtype=np.str_)
R

array(['病人', '头疼', '肌肉疼', '体温', '流感'], dtype='<U3')

#### 论域是一个对象$ U $
* 论域其实就是平时建模时候的 X 变量矩阵
* 在信息系统中，这就是 U ，是非空有限对象集，称为论域
* $ U = \{e1,e2,e3,e4,e5,e6\} $ [e1,e2 ……]这些其实是论域的标签

In [6]:
U = np.matrix(table.iloc[:,:],dtype=np.str_)
U

matrix([['e1', '是', '是', '正常', '否'],
        ['e2', '是', '是', '高', '是'],
        ['e3', '是', '是', '很高', '是'],
        ['e4', '否', '是', '正常', '否'],
        ['e5', '否', '否', '高', '否'],
        ['e6', '否', '是', '很高', '是']], dtype='<U2')

####  $ V $ 是属性值的集合

In [7]:
V = np.unique(np.ravel(U[:,1:]))
V

array(['否', '很高', '是', '正常', '高'], dtype='<U2')

#### $ V_{a} $ 就是各属性的值域，白话说就是各属性的值的取值范围
##### 比如这张表里:
* $ V_{头疼} $ ：的取值范围就是 {是，否}
* $ V_{肌肉疼 } $ ： 的取值范围就是 {是，否}
* $ V_{体温} $ ： 的取值范围就是 {正常，高，很高}

In [8]:
def VaRange(U,R):
    return_eq = {"变量名":[],"值域":[]}
    # 切分出值矩阵
    Vmatrix = U[:,1:]
    # 且分出列明
    Rs = R[1:]

    for R_name,Varray in zip(Rs,Vmatrix):
        # 先将列数据降维到行数据
        lower_dimensional = np.ravel(Varray)
        # 对数据去重得到 Va 也就是属性的值域
        Va = np.unique(lower_dimensional) 
        return_eq["变量名"].append(R_name)
        return_eq["值域"].append(Va)

    return pd.DataFrame(return_eq)

VaRange(U,R)

,变量名,值域
0,头疼,"[否, 是, 正常]"
1,肌肉疼,"[是, 高]"
2,体温,"[很高, 是]"
3,流感,"[否, 是, 正常]"


$f$ : 是 $U \times R \rightarrow V$ 的一个信息函数, 它为每个对象 $\mathbf{x}$ 的 每个属性 $\mathbf{a}$ 赋予一个属性值, 即
$$
a \in R, x \in U, f_a(x) \in V_a
$$
* 比如 $ e1 $ 在这里就是 【头疼】这一项 映射的值就是【是】，这个过程就是这个信息函数的作用  
* 这里调用f函数的要求是： $ x \in U $ $ a \in R $ 
* 返回的 $f(x) \in V$

In [9]:
def f(U,R,x,a):
    '''
    信息函数
    '''
    a = np.array(a)
    x = np.array(x)
    # 切分出 U 的标签 e_{x}
    U_i = U[:,0]

    # 生成 U 轴的布尔值索引
    m,n = U_i.shape
    x_index = np.zeros(m)
    for x_i in x: 
        x_index += np.ravel(U_i==x_i)
    # 将x所属U的位置数字变为bool值索引 
    x_index = x_index.astype(bool)

    # 生成 R 轴的布尔值索引
    m = R.size
    a_index = np.zeros(m)
    for a_i in a:
        a_index += np.ravel(R == a_i)
    # 将a所属R的位置数字变为bool值索引 
    a_index = a_index.astype(bool)

    # 切分出当前信息
    fx = U[x_index].T[a_index].T
    return pd.DataFrame(data=fx,columns=a,index=x)

f(U,R,a=["头疼","体温"],x=["e1","e2"])

,头疼,体温
e1,是,正常
e2,是,高


#### IND(A) 给定A列表对对象$e_{i}$等价关系进行划分
* 大白话说等价关系就是两或者多个对象,在某一个或着多个属性上相同，那么就说他们之间是等价关系
* A 可能包含一个属性，或者多个属性
* 参数必须满足条件 $ A \in R $
* 因为存在自反性，所以对象自己和自己也是一个等价类

In [10]:
def IND(U,R,A:iter,out_dataframe=None):
    """
    给定属性名称，查询是否存在等价类

    Args:
        U (_type_): _description_
        R (_type_): _description_
        A (iter): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    # 切分出 U 的标签 e_{x}
    U_i = U[:,0]
    m = R.size
    A = np.sort(np.array(A,dtype=np.str_))
    a_index = np.zeros(m)
    for a_i in A:
        a_index += np.ravel(R == a_i)
    # 将a所属R的位置数字变为bool值索引 
    a_index = a_index.astype(bool)

    # 切出待比较的列
    U_ij = U.T[a_index].T
    # 去重相同属性
    de_duplicate_Uij = np.unique(U_ij,axis=0)


    A_length = len(A)
    return_eq = {"等价属性列":[],"等价属性":[],"等价类对象":[]}
    for de_ in de_duplicate_Uij:
        x_index = np.where(U_ij == de_,True,False).sum(axis=1)==len(A)
        return_eq["等价属性列"].append(A)
        return_eq["等价属性"].append(de_)
        return_eq["等价类对象"].append(np.ravel(U_i[x_index]))
        # print(de_,np.ravel(U_i[x_index]))
    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq

print(IND(U,R,A=["肌肉疼","头疼"],out_dataframe=True))
print(IND(U,R,A=["体温"],out_dataframe=True))

       等价属性列    等价属性         等价类对象
0  [头疼, 肌肉疼]  [否, 否]          [e5]
1  [头疼, 肌肉疼]  [否, 是]      [e4, e6]
2  [头疼, 肌肉疼]  [是, 是]  [e1, e2, e3]
  等价属性列  等价属性     等价类对象
0  [体温]  [很高]  [e3, e6]
1  [体温]  [正常]  [e1, e4]
2  [体温]   [高]  [e2, e5]


#### 查询是否是等价类
* 输入参数必须满足条件如下：
* * $ A \in R $  and $ X \in U_{name}$ 

In [55]:
def isIND(U,R,A:iter,X:iter,out_dataframe=None):
    """
    查询的是否存在等价关系
    Args:
        U (_type_): _description_
        R (_type_): _description_
        A (iter): _description_
        X (iter): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    U_i = U[:,0]
    step_1 = IND(U,R,A)
    print(step_1)
    
    A = np.sort(np.array(A,dtype=np.str_))
    X = np.sort(np.array(X,dtype=np.str_))

    Equivalence_class_object = step_1["等价类对象"]
    Equivalence_attribute = step_1["等价属性"]
    Equivalencet_attribute_columns = step_1["等价属性列"]
    

    return_eq = {
        "X":[X],"A":[A],
        "等价属性列":[]
        ,"等价属性":[]
        ,"等价类对象":[]
            }
    
    
    for _,x in enumerate(Equivalence_class_object):
        # 求X对象名与等价类中的对象名的交集
        is_intersection = np.intersect1d(X ,x)
        # 如果存在交集，就存储
        if len(is_intersection) >= len(X):
            
            return_eq["等价类对象"].append(Equivalence_class_object[_])
            return_eq["等价属性"].append(Equivalence_attribute[_])
            return_eq["等价属性列"].append(Equivalencet_attribute_columns[_])
    
    
    return_condition = len(return_eq["等价属性列"])>0
    if out_dataframe:
        if return_condition:
            return pd.DataFrame(return_eq)
        return f"查询属性A:{A}，与查询对象X:{X}，没有等价类"
    else:
        if return_condition:
            return return_eq
        return f"查询属性A:{A}，与查询对象X:{X}，没有等价类"
    

isIND(U,R,A=["头疼","肌肉疼"],X=["e4","e6"],out_dataframe=True)

{'等价属性列': [array(['头疼', '肌肉疼'], dtype='<U3'), array(['头疼', '肌肉疼'], dtype='<U3'), array(['头疼', '肌肉疼'], dtype='<U3')], '等价属性': [array(['否', '否'], dtype='<U2'), array(['否', '是'], dtype='<U2'), array(['是', '是'], dtype='<U2')], '等价类对象': [array(['e5'], dtype='<U2'), array(['e4', 'e6'], dtype='<U2'), array(['e1', 'e2', 'e3'], dtype='<U2')]}


,X,A,等价属性列,等价属性,等价类对象
0,"[e4, e6]","[头疼, 肌肉疼]","[头疼, 肌肉疼]","[否, 是]","[e4, e6]"




#### 上近似和下近似
* 对象 $ X \in U $ 
* $ R_{obj} 为 给定 $ A $ 属性后用 $ IND(A) $  求出的等价类集表 $
* 上下近似是对象集 X 和 R 等价类集表之间的关系

* 定义一：
* * 下近似集是在那些所有的包含于X 的知识库中的集合中求并得到的（包含在X内的最大可定义集）
* * 上近似则是将那些包含X的知识库中的集合求并得到的（包含X的最小可定义集）

* 定义二：

* * 下近似包含了所有使用知识R可确切分类到X的元素。
* * 上近似包含了所有那些可能是属于X的元素，

* 定义三：

* * 下近似集定义为：根据现有知识R，判断U中所有肯定属于X的对象所组成的集合，即，式中，表示等价关系R下包含关系x的等价类；
* * 上近似集定义为：根据现有知识R，判断U中一定属于和可能属于X的对象所组成的集合。

* 定义四：

* * 下近似：最小上界
* * 上近似：最大下界。
————————————————
版权声明：本文为CSDN博主「woodman77」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/qq_42984090/article/details/126049825

In [12]:
def lower_approximation(U,X,R,A,out_dataframe=None):
    """
    下近似

    Args:
        X (_type_): _description_
        Robj (_type_): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    Robj = IND(U,R,A)
    # A = np.sort(np.array(A,dtype=np.str_))
    X = np.sort(np.array(X,dtype=np.str_))

    Equivalence_class_object = Robj["等价类对象"]
    Equivalence_attribute = Robj["等价属性"]
    Equivalencet_attribute_columns = Robj["等价属性列"]

    return_eq =[]
    
    for eco in Equivalence_class_object:
        m = len(eco)
        ### 如果eco是X的子集
        is_subsets = np.in1d(eco,X).sum()==m
        if is_subsets:
            return_eq.append(eco)
    ### 求所有是X子集的eco集合的并集
    return_eq = {"X":[X],"A":[A],"下近似":[np.unique(return_eq)]}

    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq


X_case = ["e1","e2","e3","e4"]
print(f"X:\n{X_case}")
A=["肌肉疼","头疼"]
print(f"A:\n{A}")
lower_approximation(U=U,X=X_case,R=R,A=A,out_dataframe=True)


X:
['e1', 'e2', 'e3', 'e4']
A:
['肌肉疼', '头疼']


,X,A,下近似
0,"[e1, e2, e3, e4]","[肌肉疼, 头疼]","[e1, e2, e3]"


In [13]:
def upper_approximation(U,X,R,A,out_dataframe=None):
    """上近似

    Args:
        X (_type_): _description_
        Robj (_type_): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    Robj = IND(U,R,A)
    
    X = np.sort(np.array(X,dtype=np.str_))

    Equivalence_class_object = Robj["等价类对象"]
    Equivalence_attribute = Robj["等价属性"]
    Equivalencet_attribute_columns = Robj["等价属性列"]

    return_eq =[]
    
    for eco in Equivalence_class_object:
        ### 求eco与X的交集
        intersection_set = np.intersect1d(eco,X)
        if len(intersection_set)>0:
            ### 将 eco与X的交集插入
            for e in eco:
                return_eq.append(e)
    ### 求eco与X的交集的并集
    return_eq = {"X":[X],"A":[A],"上近似":[np.unique(return_eq)]}

    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq

X_case = ["e1","e2","e3","e4"]
print(f"X:\n{X_case}")
A=["肌肉疼","头疼"]
print(f"A:\n{A}")
upper_approximation(U=U,X=X_case,R=R,A=A,out_dataframe=True)

X:
['e1', 'e2', 'e3', 'e4']
A:
['肌肉疼', '头疼']


,X,A,上近似
0,"[e1, e2, e3, e4]","[肌肉疼, 头疼]","[e1, e2, e3, e4, e6]"


#### 正域
$ Pos(X) = A(X) $

$ A(X) $：就是下近似

In [14]:
def Pos_A(U,X,R,A,out_dataframe=None):
    """正域

    Args:
        U (_type_): _description_
        X (_type_): _description_
        R (_type_): _description_
        A (_type_): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    A_lower = lower_approximation(U=U,X=X,R=R,A=A)['下近似']
    return_eq = {"X":[X],"A":[A],"正域":[A_lower]}
    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq

X_case = ["e1","e2","e3","e4"]
print(f"X:\n{X_case}")
A=["肌肉疼","头疼"]
print(f"A:\n{A}")
Pos_A(U=U,X=X_case,R=R,A=A,out_dataframe=True)

X:
['e1', 'e2', 'e3', 'e4']
A:
['肌肉疼', '头疼']


,X,A,正域
0,"[e1, e2, e3, e4]","[肌肉疼, 头疼]","[[e1, e2, e3]]"


#### 负域

$  NEG_{A}(X) = U-A^{-}(X) $ 求U与 上近似 的差集

$ A^{-}(X) $ :为上近似

In [15]:
def NEG_A(U,X,R,A,out_dataframe=None):
    """负域

    Args:
        U (_type_): _description_
        X (_type_): _description_
        R (_type_): _description_
        A (_type_): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    A_upper = upper_approximation(U=U,X=X,R=R,A=A)["上近似"]
    U_i = np.ravel(U[:,:1])
    # 求U-上近似A-(x)的差集
    NEGx = np.setdiff1d(U_i,A_upper)

    return_eq = {"X":[X],"A":[A],"负域":[NEGx]}
    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq

X_case = ["e1","e2","e3","e4"]
print(f"X:\n{X_case}")
A=["肌肉疼","头疼"]
print(f"A:\n{A}")
NEG_A(U=U,X=X_case,R=R,A=A,out_dataframe=True)

X:
['e1', 'e2', 'e3', 'e4']
A:
['肌肉疼', '头疼']


,X,A,负域
0,"[e1, e2, e3, e4]","[肌肉疼, 头疼]",[e5]


### 边界

$ BND_{A}(X) = A^{-}(X) - A_{-}(X)$ 求上近似与下近似的差集

$ A^{-}(X) $ 上近似

$ A_{-}(X) $ 下近似

In [16]:
def BND_A(U,X,R,A,out_dataframe=None):
    """边界

    Args:
        U (_type_): _description_
        X (_type_): _description_
        R (_type_): _description_
        A (_type_): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    A_upper = upper_approximation(U=U,X=X,R=R,A=A)["上近似"]
    A_lower = lower_approximation(U=U,X=X,R=R,A=A)['下近似']
    BNGx = np.setdiff1d(A_upper,A_lower)
    
    return_eq = {"X":[X],"A":[A],"边界":[BNGx]}
    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq

X_case = ["e1","e2","e3","e4"]
print(f"X:\n{X_case}")
A=["肌肉疼","头疼"]
print(f"A:\n{A}")
BND_A(U=U,X=X_case,R=R,A=A,out_dataframe=True)

X:
['e1', 'e2', 'e3', 'e4']
A:
['肌肉疼', '头疼']


,X,A,边界
0,"[e1, e2, e3, e4]","[肌肉疼, 头疼]","[e4, e6]"


#### 粗糙集定义：
* $ A^{-}(X) \neq A_{-}(X)$ ： 如果上近似不等于下近似 就说X 是 A的粗糙集
* $ A \in R $  

In [41]:
def isRoughSet(U,X,R,A,out_dataframe=None):
    """是否是粗糙集

    Args:
        U (_type_): _description_
        X (_type_): _description_
        R (_type_): _description_
        A (_type_): _description_
        out_dataframe (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    A_upper = upper_approximation(U=U,X=X,R=R,A=A)["上近似"][0]
    A_lower = lower_approximation(U=U,X=X,R=R,A=A)['下近似'][0]
    is_equality = set(A_upper) == set(A_lower)
    
    BND_a =BND_A(U=U,X=X,R=R,A=A)["边界"]
    BNDisEmptySet = len(BND_a) == 0
    
    return_eq = {
        "X":[X]
        ,"A":[A]
        ,"粗糙/精确集":is_equality & BNDisEmptySet and  "X为A的精确集" or "X为A的粗糙集"
    }
    if out_dataframe and out_dataframe != bool:
        return pd.DataFrame(return_eq)
    elif out_dataframe == bool:
        return is_equality==False

    return return_eq
    
    
X_case = ["e1","e2","e3","e4"]
print(f"X:\n{X_case}")
A=["肌肉疼","头疼"]
print(f"A:\n{A}")    
print(f"isRoughSet 返回 是否是粗糙集 ：{isRoughSet(U=U,X=X_case,R=R,A=A,out_dataframe=bool)}")
print(f"isRoughSet 返回 字典数据 ：{isRoughSet(U=U,X=X_case,R=R,A=A,out_dataframe=False)}")
isRoughSet(U=U,X=X_case,R=R,A=A,out_dataframe=True)

X:
['e1', 'e2', 'e3', 'e4']
A:
['肌肉疼', '头疼']
False
isRoughSet 返回 是否是粗糙集 ：True
False
isRoughSet 返回 字典数据 ：{'X': [['e1', 'e2', 'e3', 'e4']], 'A': [['肌肉疼', '头疼']], '粗糙/精确集': 'X为A的粗糙集'}
False


,X,A,粗糙/精确集
0,"[e1, e2, e3, e4]","[肌肉疼, 头疼]",X为A的粗糙集
